# Distillation 

Ce a pour but d'entrainer un MPS multicouche à l'aide d'un réseau complètement connecté

In [ ]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jemisjoky/TorchMPS.git to /tmp/pip-req-build-sqr6zq4p
  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git /tmp/pip-req-build-sqr6zq4p
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py) ... done
  Created wheel for torchmps: filename=torchmps-0.1.0-py3-none-any.whl size=59683 sha256=3484227154570d942390eef9b0b46ec1974e39f70446eee9b45985b79ae2bda3
  Stored in directory: /tmp/pip-ephem-wheel-cache-u9tpvf41/wheels/11/91/cf/b491a766df84816f76afc5c41a6dc200f736dc47c9c38f5b0e
Successfully built torchmps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 23.9 MB/s eta 0:00:00


In [ ]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train = 2000
nb_test = 500
chosen_bs = 100
input_size = 28*28
nb_classes = 10

# Teacher hyperparameters
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 20
hidden_size = 70
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9

# Student hyperparameters
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False
feature_map = lambda x : torch.tensor([1, x]).to(chosen_device)
feature_dim = 2
# Training parameters
gauss_epochs = 5 # number of epochs with added gaussian noise
gn_var = 0.3 #added gaussian noise variance
gn_mean = 0 #added gaussian noise mean
n_epochs_lmps = 10
mps_learn_rate = 0.0001
mps_reg = 0.0
mps_chosen_loss = nn.CrossEntropyLoss().to(chosen_device)

In [ ]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, 
    sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets',
 train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, 
    sampler = test_subset, batch_size = chosen_bs)

In [ ]:

# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, hidden_size)
        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.lin3 = nn.Linear(hidden_size, hidden_size)
        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.lin5 = nn.Linear(hidden_size, hidden_size)
        self.lin6 = nn.Linear(hidden_size, 10)

    def middleforward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        return y

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.Adam(fcnn_teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(loss.item())

# get accuracy
# Get the validation set classification accuracy
teacher_acc_score = 0
teacher_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    teacher_acc_score += x_mb.size()[0] * teacher_acc_metric( fcnn_teacher(x_mb), y_mb )
print("The teacher's accuracy score is:")
print(teacher_acc_score / nb_test) #divide by total size



2.234989881515503
1.5901509523391724
1.1908395290374756
1.1410343647003174
0.9046189785003662
0.7646516561508179
0.8461199402809143
0.8103229403495789
0.87164306640625
0.7510700821876526
0.712493360042572
0.5702184438705444
0.7493656873703003
0.7436438798904419
0.6074837446212769
0.5503943562507629
0.6597182750701904
0.8016920685768127
0.6493374109268188
0.5768996477127075
The teacher's accuracy score is:
tensor(0.7286, device='cuda:0')


In [ ]:

# Initialize the MPS modules
student_mps1 = MPS(
    input_dim = input_size,
    feature_dim = feature_dim,
    output_dim = hidden_size,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps1.register_feature_map(feature_map)

student_mps2 = MPS(
    input_dim = hidden_size, #crucial setting
    feature_dim = feature_dim,
    output_dim = nb_classes,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps2.register_feature_map(feature_map)

def student(x):
    return student_mps2( student_mps1(x) )


# Instantiate the optimizer and softmax
lmps1_optimizer = torch.optim.Adam(student_mps1.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)

lmps2_optimizer = torch.optim.Adam(student_mps2.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)

softmax = nn.Softmax(dim=1)

# Training loop for the student_mps1
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        # Added gaussian noise
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get softmax of output of teacher in middlelayer!
        y_mb = (softmax(fcnn_teacher.middleforward(x_mb))).to(chosen_device)

        # Foward propagation
        y_hat_mb = student_mps1(x_mb)
        loss = mps_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps1_optimizer.step()
        lmps1_optimizer.zero_grad()

    print(loss.item())

# Training loop for the student_mps2
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        # Added gaussian noise
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get softmax of output of teacher
        y_mb = ( softmax(fcnn_teacher(x_mb)) ).to(chosen_device)

        # Foward propagation
        y_hat_mb = student_mps1(x_mb)
        # Apply second forward propagation on first output
        y_hat_mb = student_mps2(y_hat_mb)
        loss = mps_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps2_optimizer.step()
        lmps2_optimizer.zero_grad()

    print(loss.item())



# Get the validation set classification accuracy
student_acc_score = 0
student_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    student_acc_score += x_mb.size()[0] * student_acc_metric( student(x_mb), y_mb )
print("Student_acc_score:")
print(student_acc_score / nb_test) #divide by total size


4.248688697814941
4.175558567047119
4.126443386077881
4.240897178649902
4.246966361999512
4.2449727058410645
4.241263389587402
4.240814208984375
4.218619346618652
4.171491622924805
2.3025825023651123
2.301757335662842
2.270613670349121
2.560258388519287
2.302800178527832
2.3027331829071045
2.3019351959228516
2.2326111793518066
1.8474551439285278
2.244260549545288
Student_acc_score:
tensor(0., device='cuda:0')
